### Import and global settings

In [10]:
import seaborn as sn
import numpy as np
import pandas as pd
import json
import csv
import calendar
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    BaseEnsemble,
    RandomForestRegressor,
    RandomTreesEmbedding,
    ExtraTreesClassifier,
    ExtraTreesRegressor,
    BaggingClassifier,
    BaggingRegressor,
    IsolationForest,
    GradientBoostingRegressor,
    AdaBoostRegressor,
    VotingClassifier,
    VotingRegressor,
    StackingClassifier,
    StackingRegressor,
    HistGradientBoostingClassifier,
    HistGradientBoostingRegressor
)
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from typing import Dict, List, Tuple, Any
from collections import Counter

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1)

#the time period constant used to calculate the EMA
DAYS_FOR_EMA = 21
#the time period constant used to calculate the RSI
DAYS_FOR_RSI = 14

#constant used to determine the length of each period to get the average of close price
CLOSE_PRICE_MEAN_DAYS = 7

#period length used fo MA volume calculation
MA_VOLUME_DAYS = 14

RANDOM_STATE = 45

TEST_SIZE = 0.1251353

In [ ]:
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    BaseEnsemble,
    RandomForestRegressor,
    RandomTreesEmbedding,
    ExtraTreesClassifier,
    ExtraTreesRegressor,
    BaggingClassifier,
    BaggingRegressor,
    IsolationForest,
    GradientBoostingRegressor,
    AdaBoostRegressor,
    VotingClassifier,
    VotingRegressor,
    StackingClassifier,
    StackingRegressor,
    HistGradientBoostingClassifier,
    HistGradientBoostingRegressor
)

### Pull data

In [3]:
'''

 https://www.kaggle.com/datasets/kannan1314/amazon-stock-price-all-time
 Basic CSV file with open, high, low, close, adjusted close price and volume of Amazon stock for each day.

 https://www.kaggle.com/datasets/henryshan/amazon-com-inc-amzn
 The dataset includes the daily Amazon.com, Inc. stock price. (BU KULLANILMAYABİLİR)


 https://www.kaggle.com/datasets/varpit94/amazon-stock-data
 Amazon (AMZN) historic prices (USD), daily data.

https://www.kaggle.com/datasets/beeru999/amazon-stock-prices1997-2021
This CSV file contains historical stock price data for Amazon (AMZN) from May 15, 1997, to April 5, 2023. The dataset includes the following columns:

Date: The date of the trading day.
Open: The opening price of the stock on that day.
High: The highest price the stock reached during the trading day.
Low: The lowest price the stock reached during the trading day.
Close: The closing price of the stock on that day.
Adj Close: The adjusted closing price, which accounts for dividends, stock splits, and other adjustments.
Volume: The number of shares traded during the day.
'''
#df1 = pd.read_csv('data/Amazon.csv')
#df2 = pd.read_csv('data/AMZN.csv')
#df3 = pd.read_csv('data/AMZN (1).csv')
#TODO şimdilik tek dataset, datalar farklı kafa karışmasın
df4 = pd.read_csv('data/AMZN (2).csv')



In [9]:
#display(df1.iloc[-1:])
#display(df2.iloc[-1:])
#display(df3.iloc[-1:])
display(df4)

np.False_

### Process data

In [ ]:
'''
DATALARDA HER GÜN BULUNMUYOR

RSI EMA falan hesaplarken timedelta değil index bakalım (ya da uygunsuz mu olur)
'''

#TODO MACD sonra bak, hesaplayabilmek için 12 günlük ve 26 günlük EMA da gerekiyor
#TODO volatility kısmı bak (yapılmalı mı?)
#TODO günden güne volume kısmında yüzdelik kısmına bak (pzts salı %40'tan %60'a çıkmış vs.) (buna bakılabilir mi?)

#TODO önceki günleri de hesaba kat
#TODO direk close tahmin etmek yerine, yukarı mı gidecek aşağı mı gidecek onlara bak
#TODO close, volume veya diğer fiyatla ilgilli özelliklere log transofrmation yap varience stabilizasyonu için
#TODO scaling yap

#TODO train test bölerken son günler test olsun


def is_month_start(row: Dict) -> bool:
    """
    Check if the given date is the start of the month.

    Args:
    row (dict): A dictionary containing a 'Date' key with the date string.

    Returns:
    bool: True if the date is the end of the month, False otherwise.
    """
    return row['Date'].split('-')[2] == '01'

def is_month_end(row: Dict) -> bool:
    """
    Check if the given date is the end of the month.

    Args:
    row (dict): A dictionary containing a 'Date' key with the date string.

    Returns:
    bool: True if the date is the end of the month, False otherwise.
    """
    #diğeri gibi yapmadık, 28-29 Şubat'ı da dahil edebilmek için
    date_str = row['Date']
    date = datetime.strptime(date_str, "%Y-%m-%d")
    last_day = calendar.monthrange(date.year, date.month)[1]
    return date.day == last_day

df4["month_start"]=df4.apply(is_month_start, axis=1, inplace=True)
df4["month_end"]=df4.apply(is_month_end, axis=1, inplace=True)

df4["high_low_diff"] = df4["High"] - df4["Low"]

df4["open_close_diff"] = df4["Open"] - df4["Close"]

df4["volume*Close"] = df4["Volume"] * df4["Close"]

'''DATA HER BİR GÜNÜ İÇERMİYOR, 7 GÜNLÜK ORTALAMA VS. MANTIKLI MI?'''

#14 gün de olabilir
def get_close_avg(row: Dict, days: int, df: pd.DataFrame) -> float:
    """
    Get the average of the closing prices of the previous 7 days.

    Args:
    row (dict): A dictionary containing a 'Date' key with the date string.

    Returns:
    float: The average of the closing prices of the previous 7 days.
    """
    #get the previous 7 days
    if row.name < 7:
        return 0
    else:
        index = row.name
        close_avg = df.loc[index-days:index]["Close"].mean()
        return close_avg

df4[f"close_price_mean_{CLOSE_PRICE_MEAN_DAYS}_days"] = df4.apply(lambda row: get_close_avg(row, CLOSE_PRICE_MEAN_DAYS, df4), axis=1, inplace=True)

def calculate_ema(row: Dict, ema_period: int, df: pd.DataFrame) -> float:
    """
    Calculate the Exponential Moving Average (EMA) for a given row.

    Args:
    row (dict): A dictionary containing the row data.
    ema_period (int): The number of periods to consider for calculating EMA.

    Returns:
    float: The EMA value for the given row.
    """
    #for initial prices, return the average of close prices over n number of periods (a simple moving average)
    if row.name < ema_period:
        return df4.loc[:row.name, 'Close'].mean()
    #for others, calculate the EMA normally
    else:
        previous_ema = df4.loc[row.name - 1, f'ema_{ema_period}']
        close_price = row['Close']
        smoothing_constant = 2 / (ema_period + 1)
        ema = (close_price - previous_ema) * smoothing_constant + previous_ema
        return ema

df4[f'ema_{DAYS_FOR_EMA}'] = df4.apply(lambda row: calculate_ema(row, DAYS_FOR_EMA, df4), axis=1)

#RSI için ilk 21 gün olmuyor, ilk 21 günü atla
def calculate_RSI(df: pd.DataFrame, row: Dict, rsi_period: int) -> float:

    '''Calculates the RSI for given row and returns it
    Args:
    df: dataframe used (added as parameter, global variables can be buggy thats why)
    row (dict): A dictionary containing the row data.
    rsi_period (int): The number of periods to consider for calculating RSI.

    Returns:
    float: The RSI value for the given row. 
    '''
    #get the index of row (we dont have each day thats why we dont use timedelta)
    index = row.name
    
    #close prices of previous 21 days
    prev_close = df.iloc[index-rsi_period:index]

    def calculate_difference(row: Dict, df: pd.DataFrame) -> float:
        '''Calculates the difference between close prices.
        Returns zero for the first day
        Args:
        row (dict): a row in dataframe
        df (DataFrame): dataframe

        Returns:
            diff (float): the difference between two rows close prices. Zero if first day
        '''
        #first day
        if row.name == 0:
            return 0
        #-1 to get the previous day
        index = row.name - 1
        diff = row["Close"] - df.loc[index]["Close"]

        return diff

    #don't include the first day for RSI (cant calculate the difference for first day)
    prev_close["diff"] = prev_close.apply(calculate_difference, axis=1)
    
    #TODO check this calculation
    avg_gains = prev_close[prev_close["diff"] > 0]["diff"].mean()
    avg_losses = prev_close[prev_close["diff"] < 0]["diff"].mean()
    
    #calculate and return
    rsi = avg_gains / avg_losses
    return rsi

df4[f"RSI_{DAYS_FOR_RSI}_days"] = df4.apply(lambda row: calculate_RSI(df4, row, DAYS_FOR_RSI), axis=1)

def calculate_liquidity(row: Dict) -> float:
    '''Calculates the liquidity for the given row
    volume / close
    Args:
    row (dict): a row of the dataframe

    Returns:
        the liquidity of the row
    '''
    return row["Volume"] / row["Close"]

df4["liquidity"] = df4.apply(calculate_liquidity, axis=1)

def calculate_MA_volume(row: Dict, days: int, df: pd.DataFrame) -> float:
    '''Calculates the volume moving average over a given period of time
    Args:
    row (dict): a row of dataframe
    days (int): the length of a period
    df (Dataframe): dataframe to operate on
    '''

    if row.name < days:
        return 0
    else:
        index = row.name
        avg = df.loc[index-days: index]["Volume"].mean()
        return avg
    
df4[f"volume_MA_{MA_VOLUME_DAYS}_days"] = df4.apply(calculate_MA_volume, axis=1)



### Model choose and fit

In [ ]:
'''

scheduler kullan

linear regression

from r_model import Ridge

CNN

dropout

dense

mse kullanıldı genelde

birden fazla model

TRACK transformer model

loss plateau

her okuduğunda datada perturbation yap

pytoerch yoerine tensorflow kullanmak daha etkili olabilir

hipotez testi

Man-Whittney U test

VBO

greed search

gridf

randomizedCV search

torch.inference_mode

LSTM modülü

GRU modülü

tensor çevir

denormalize data

fakrlı grafikler çizdir

seed model
'''

X = df4.columns.tolist()
y = np.stack(df4["Close"])

x_train, y_train, x_test, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

# Creating models for each classifier

def define_models() -> List[Any]:
    '''Defines the classifier models and returns them in a list
    Args:
    None

    Returns:
    models (list): a list of models
    '''
    # RandomForestClassifier
    rf_model = RandomForestClassifier()

    # GradientBoostingClassifier
    gb_model = GradientBoostingClassifier()

    # AdaBoostClassifier
    ab_model = AdaBoostClassifier()

    # ExtraTreesClassifier
    et_model = ExtraTreesClassifier()

    # BaggingClassifier
    bagging_model = BaggingClassifier()

    # IsolationForest
    if_model = IsolationForest()

    # VotingClassifier
    voting_model = VotingClassifier(estimators=[])

    # StackingClassifier
    stacking_model = StackingClassifier(estimators=[])

    # HistGradientBoostingClassifier
    hist_gb_model = HistGradientBoostingClassifier()

    models = [
        rf_model,
        gb_model,
        ab_model,
        et_model,
        bagging_model,
        if_model,
        voting_model,
        stacking_model,
        hist_gb_model
    ]

    return models

models: list = define_models()

#TODO ilk etapta k-cross validation yapmayacağız, sonra eklenecek

#dictionary seçtik histogram plot yaparken kolaylık olsun diye
histogram_data: Dict = {}
for m in models:
    m.fit(x_train, y_train)
    y_pred = m.predict(x_test)
    score = accuracy_score(y_test, y_pred)
    model_name = m.__class__.__name__
    histogram_data[model_name] = score


# Extracting model names and accuracy scores
model_names = list(histogram_data.keys())
accuracy_scores = list(histogram_data.values())


### Print results

In [ ]:
def print_results(model_names, scores):
    # Plotting the histogram
    plt.figure(figsize=(10, 6))
    plt.bar(model_names, accuracy_scores, color='skyblue')
    plt.xlabel('Model Names')
    plt.ylabel('Accuracy Scores')
    plt.ylim(0.0, 1.0)
    plt.title('Model Accuracy Scores')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()


print_results(model_names, accuracy_scores)